In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8')

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print "%s\t%s\t%d" % (word.lower(), word, 1)

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys
import re

current_key = None
word_condition_sum = 0
word_uncondition_sum = 0

pattern = re.compile("^[A-Z](?=[a-z0-9])")

for line in sys.stdin:
    try:
        key, origin, count = line.strip().split('\t', 2)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            total = word_condition_sum + word_uncondition_sum
            uncondition_persentage = float(word_uncondition_sum) / total * 100.
            if uncondition_persentage < 0.5:
                print "%s\t%d" % (current_key, total)
        word_condition_sum = 0
        word_uncondition_sum = 0
        current_key = key
    if bool(pattern.match(origin)):
        word_condition_sum += count
    else:
        word_uncondition_sum += count

if current_key:
    total = word_condition_sum + word_uncondition_sum
    uncondition_persentage = float(word_uncondition_sum) / total * 100.
    if current_key and uncondition_persentage < 0.5:
        print "%s\t%d" % (current_key, total)

Overwriting reducer.py


In [3]:
%%writefile mapper-words-ranking.py

import sys

for line in sys.stdin:
    try:
        w, c = line.strip().split('\t', 1)
    except ValueError as e:
        continue
    print "%d\t%s" % (int(c), w)

Overwriting mapper-words-ranking.py


In [4]:
%%writefile reducer-words-ranking.py

import sys

for line in sys.stdin:
    try:
        c, w = line.strip().split('\t', 1)
    except ValueError as e:
        continue
    print "%s\t%s" % (w, c)

Overwriting reducer-words-ranking.py


In [5]:
%%bash

OUT_DIR_1="name_count_result_lkv"
NUM_REDUCERS_1=8

OUT_DIR_2="words_ranking_result_lkv"
NUM_REDUCERS_2=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR_1} > /dev/null
hdfs dfs -rm -r -skipTrash ${OUT_DIR_2} > /dev/null


yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming name count" \
    -D mapreduce.job.reduces=${NUM_REDUCERS_1} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR_1} > /dev/null


yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
     -D mapred.jab.name="Streaming word ranking" \
     -D mapreduce.job.reduces=${NUM_REDUCERS_2} \
     -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
     -D mapred.text.key.comparator.options=-nr\
     -files mapper-words-ranking.py,reducer-words-ranking.py \
     -mapper "python mapper-words-ranking.py" \
     -reducer "python reducer-words-ranking.py" \
     -input ${OUT_DIR_1}/part* \
     -output ${OUT_DIR_2} > /dev/null
     
hdfs dfs -cat ${OUT_DIR_2}/part-00000 | head -5 | tail -1

french	5753


18/01/14 19:18:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/14 19:18:51 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/01/14 19:18:52 INFO mapred.FileInputFormat: Total input files to process : 1
18/01/14 19:18:52 INFO mapreduce.JobSubmitter: number of splits:2
18/01/14 19:18:52 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1515912325866_0013
18/01/14 19:18:52 INFO impl.YarnClientImpl: Submitted application application_1515912325866_0013
18/01/14 19:18:52 INFO mapreduce.Job: The url to track the job: http://4c3b86566330:8088/proxy/application_1515912325866_0013/
18/01/14 19:18:52 INFO mapreduce.Job: Running job: job_1515912325866_0013
18/01/14 19:18:58 INFO mapreduce.Job: Job job_1515912325866_0013 running in uber mode : false
18/01/14 19:18:58 INFO mapreduce.Job:  map 0% reduce 0%
18/01/14 19:19:14 INFO mapreduce.Job:  map 46% reduce 0%
18/01/14 19:19:20 INFO mapreduce.Job:  map 67% reduce 0%
18/01/14 19:19:24 INFO 